In [ ]:
# setup
from IPython.display import display,HTML
display(HTML('<style>.prompt{width: 0px; min-width: 0px; visibility: collapse}</style>'))
display(HTML(open('../rise.css').read()))

# imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="whitegrid", font_scale=1.5, rc={'figure.figsize':(12, 6)})

<h1>Shuffle</h1>

Our final activity will involve building a labyrinth by following this procedure:

1. Start with a graph, embedded in the plane.
2. Choose a uniformly random permutation of the edges.
3. Use this permutation as weights for the edges.
4. With these weights, apply Kruskal's algorithm to calculate the minimal spanning tree.

Let us focus on step 2. How do we select a uniformly random permutation of a set, given the ability to select random numbers from small intervals? Here are three strategies that work and some comments about potential drawbacks:

1. Assign each edge a random number, then sort the edges.
    - We will need enough precision with the random number to avoid multiple edges being assigned the same number. This will lead to wasted randomness in calculating unused decimal places.
    - We still need to sort the numbers, which costs $O(n\log(n))$ work, where $n$ is the size of the list.
2. Choose a uniformly random number between $1$ and $n!$. Select the permutation that corresponds to this number.
    - It's not so easy to generate a random integer with a large range. If we select a random real number from the interval $[0,1]$, then we need to calculate enough decimals so that all of the integers in the range $1$ to $n!$ are represented.
    - Since there are n! permutations, there is some way to number these permutations, but we have not specified a concrete numbering. For this strategy to work, we need to specify this numbering system and an algorithm to calculate the permutation associated with its number.
3. (Fisher shuffle). Choose a uniformly random item to be the first in the permutation. Then, among the remaining items, choose a uniformly random item to be second. Continue in this way, selecting the $k^{th}$ element to be uniformly random from the set of items that have not been selected before.
    - This is the [standard algorithm](https://en.wikipedia.org/wiki/Fisher%E2%80%93Yates_shuffle), Algorithm P from Knuth's TAOCP. Since it is a standard algorithm, it is probably best.

The Fisher shuffle can be visualized as a lottery drawing. Each item is a numbered ball. The balls are put into a bag and drawn, uniformly at random, one-by-one. It's pretty clear that the resulting order of the balls will be uniformly random.

<h3>In-place shuffling</h3>

One advantage of the Fisher shuffle is that it can be performed in-place, without additional memory. It is not necessary to create a new list and move the shuffled elements from the old list into the new list. This improved implementation has the following steps.

1. Keep track of a counter, $i$, initially $1$.
2. Select an index $r_i$ between $0$ and $n-i$, uniformly at random.
3. Swap item $i-1$ with item $i-1+r_i$.
4. Increment $i$.
5. Halt if $i==n$. Otherwise, return to step 2.

<h3>Runtime Analysis.</h3>

Let's analyze the Fisher shuffle. First, we count the number of random bits used. We assume that it takes $\log(n)$ random bits to select a number from $0$ to $n-1$ uniformly at random. The number of random bits is $\log(n) +\log(n-1)+\log(n-2)+\dots+\log(2)\in O(\log(n!))\in O(n\log(n))$.

The algorithm performs $1$ swap per iteration of a loop that runs $n$ times, so the work is $O(n)$.

It can be parallelized using the scan paradigm. Generate the random bits. Create an array with $n-1$ positions. Map each position $i$ to the swap $(i-1, i-1+r_i)$. Scan the composition of these swaps, which is allowed because composition is associative. Each composition can be performed in constant span. This shows that the shuffle can be performed in $\log(n)$ span.

<h3>Proving the Fisher shuffle is uniformly random.</h3>

It is largely intuitive that the Fisher shuffle produces uniformly random permutations. Each item in the permutation is selected uniformly at random. However, it can be trick to determine whether certain algorithms truely sample from the uniform distribution. For this reason, it is useful to know what a proof of this fact looks like, to imagine how it might generalize to other sets.

<h5>Establish a direct connection between the random choices and resulting permutations.</h5>

Here is an argument that the Fisher shuffle produces uniformly random permutations. Perform the algorithm on a list $Q$. The algorithm always generates $n-1$ random numbers, where the $i^{th}$ random number $r_i$ satisfies $r_i\in [0,n-i]$. This amounts to a total of $(n)(n-1)(n-2)\dots 2=n!$ possible choices. Each choice for all $r_i$ is equally likely, because each $r_i$ is chosen independently and uniformly at random from its set. Each permutation $P$ can be generated by some choice of the numbers $r_i$, as we argue explicitly.

Explicitly, we can choose $r_i$ such that $Q_{i-1+r_i} = P_{i-1}$. Since $r_i\in [0,n-i]$, the index $i-1+r_i$ ranges over all of $[0,n-1]$, this choice for $r_i$ is well-defined and valid. To see that these $r_i$ produce $P$, note that each item of $Q$ is only swapped to the left (meaning that the index of its position decreases) once during the algorithm. To show that the algorithm puts $P_i$ in position $i$, suppose otherwise, and let $j$ be the first step that the item that is swapped with item $j-1$ is not $Q_{j-1+r_j}=P_{j-1}$ the original item at that index. For that to happen, there must be a previous swap (before step $j$) involving the position $j-1+r_j$. This would place $P_{j-1}$ in the wrong spot and would contradict $j$ being the smallest index of error. Though it is not necessary for the proof, we check the simplest case: when $j=1$, $Q_{r_1+(1-1)}=P_{0}$ by definition, so $P_0$ is correctly placed in the $0$ position.

We count that there are $n!$ choices of the $r_i$ and $n!$ permutations. The algorithm provides a function from choices to permutations. We showed explicitly that the function is onto, meaning that every permutation can be produced by some choices. We conclude that the correspondence is bijective. The probability of a permutation $P$ is the sum of the probabilities of the choices that produce it, and this is $\frac{1}{n!}$. QED

The proof establishes a direct connection between the shuffle and the bits of randomness used: every permutation has a unique set of choices $r_i$ that produce it. Using this connection, we are able to transform calculations of probability about the shuffle into calculations of probability for the random choices, which are assumed to be uniformly distributed.

<h3>Testing Distributions</h3>

Suppose that we implement the Fisher shuffle. How do we know that we've done it right? Looking at any individual output is just a sample, and there's always some probability that that any set of weird samples will happen to occur when sampling from the uniform distribution. We need a principled way to measure how likely it is that we're actually sampling from the uniform distribution.

This problem is usually solved using the $\chi$-squared [test](https://en.wikipedia.org/wiki/Chi-squared_test). The idea is to study the $\chi^2$ statistic, $\sum_{P}\frac{(f_{obs,P} - f_{exp,P})^2}{f_{exp,P}} $, where $f_{obs,P}$ is the number of times we observed outcome $P$ and $f_{exp,P}$ is the number of times we expect to see outcome $P$. We compare this statistic against the distribution of the statistic's value when the samples were actually drawn from the distribution. This allows for a $p$-value that quantifies how likely our samples were, under the assumption that they came from the uniform distribution. If it is less like than (as a standard value) $0.05$, we have evidence that suggests that the samples were not drawn from the uniform distribution.

This does not settle the question of whether or not our implementation is correct. The $\chi$ squared test only provides one way to notice when a distribution is masquarading as the uniform distribution. One case when it fails is that it only takes the number of observations into account, and not corellations between samples. The "independent uniform sample" of numbers 10101010101 would pass the $\chi^2$ test, but is obviously unlikely to be a series of independent uniform samples, due to the low chance of getting an alternating series of that length.